In [ ]:
#Modelo de Machine Learning

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# Cargar y combinar las tres partes del dataset
movies_part1 = pd.read_csv("app/data/movies_parte_1.csv")
movies_part2 = pd.read_csv("app/data/movies_parte_2.csv")
movies_part3 = pd.read_csv("app/data/movies_parte_3.csv")

# Concatenar las tres partes
movies = pd.concat([movies_part1, movies_part2, movies_part3], ignore_index=True)

# Reemplazar valores nulos en la columna 'title'
movies['title'] = movies['title'].fillna('')

# Confirmar la estructura del DataFrame
print("Dimensiones del dataset combinado:", movies.shape)
movies.head()



Dimensiones del dataset combinado: (45449, 32)


,title,release_year,overview,genres,return,languages,popularity,vote_average,vote_count,cast,...,Heavens Above!,1963,"A naive but caring prison chaplain, who happens to have the same last name as an upper class cleric, is by mistake appointed as vicar to a small and prosperous country town. His belief in charity and forgiveness sets him at odds with the conservative and narrow-minded locals, and he soon creates social ructions by appointing a black dustman as his churchwarden, taking in a gypsy family, and persuading the local landowner to provide free food for the church to distribute free to the people of the town. When the congregation leaders realise the mistake and call for the Church of England to remove him, this turns out to be a very, very difficult issue - until one clergyman realises that a British project to send a man into space is in need of an astronaut...",['Comedy'],0.0,0.484307,6.5,5.0,"[{'character': 'The Rev. John Smallwood', 'name': 'Peter Sellers'}, {'character': 'Archdeacon Aspinall', 'name': 'Cecil Parker'}, {'character': 'Lady Despard', 'name': 'Isabel Jeans'}, {'character': 'The Other Smallwood', 'name': 'Ian Carmichael'}, {'character': 'Simpson', 'name': 'Bernard Miles'}, {'character': 'Matthew Robinson', 'name': 'Brock Peters'}, {'character': 'Harry Smith', 'name': 'Eric Sykes'}, {'character': 'Rene Smith', 'name': 'Irene Handl'}, {'character': 'Winnie Smith', 'name': 'Miriam Karlin'}, {'character': 'Mrs Smith-Gould', 'name': 'Joan Miller'}, {'character': 'Major Fowler', 'name': 'William Hartnell'}, {'character': 'Fred Smith', 'name': 'Roy Kinnear'}, {'character': 'Wilson, Prison Officer (uncredited)', 'name': 'John Harvey'}, {'character': 'Housewife', 'name': 'Joan Hickson'}, {'character': 'Bank Manager', 'name': 'Eric Barker'}, {'character': 'Butcher', 'name': 'John Comer'}, {'character': 'Rev. Owen Thomas', 'name': 'Kenneth Griffith'}, {'character': 'Sir Geoffrey Despard', 'name': 'Mark Eden'}, {'character': 'Prison Governor', 'name': 'Basil Dignam'}, {'character': 'Prime Minister', 'name': 'Colin Gordon'}, {'character': 'Council Official', 'name': 'Geoffrey Hibbert'}, {'character': 'Himself', 'name': 'Ludovic Kennedy'}, {'character': 'Cleric', 'name': 'Malcolm Muggeridge'}, {'character': 'Shop Steward', 'name': 'Harry Locke'}, {'character': 'DG Assistant', 'name': 'Derek Nimmo'}, {'character': 'Deaf Man', 'name': 'Conrad Phillips'}, {'character': 'Bishop Rockeby', 'name': 'Miles Malleson'}, {'character': 'DG', 'name': 'Nicholas Phipps'}, {'character': 'Tranquilax Executive', 'name': 'Thorley Walters'}, {'character': 'Store Manager', 'name': 'Gerald Sim'}, {'character': 'Housewife', 'name': 'Anna Wing'}, {'character': 'Shopper', 'name': 'John Louis Mansi'}, {'character': '', 'name': 'John Junkin'}, {'character': '', 'name': 'Rodney Bewes'}, {'character': '', 'name': 'Steve Marriott'}, {'character': 'Tramp', 'name': 'Cardew Robinson'}]","[{'job': 'Director of Photography', 'name': 'Mutz Greenbaum'}, {'job': 'Screenplay', 'name': 'Frank Harvey'}, {'job': 'Director', 'name': 'John Boulting'}, {'job': 'Producer', 'name': 'John Boulting'}, {'job': 'Director', 'name': 'Roy Boulting'}, {'job': 'Producer', 'name': 'Roy Boulting'}, {'job': 'Editor', 'name': 'Teddy Darvas'}]"
0,Toy Story,1995.0,"Led by Woody, Andy's toys live happily in his ...","['Animation', 'Comedy', 'Family']",12.451801,['English'],21.946943,7.7,5415.0,"[{'character': 'Woody (voice)', 'name': 'Tom H...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Jumanji,1995.0,When siblings Judy and Peter discover an encha...,"['Adventure', 'Fantasy', 'Family']",4.043035,"['English', 'FranÃÂ§ais']",17.015539,6.9,2413.0,"[{'character': 'Alan Parrish', 'name': 'Robin ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Grumpier Old Men,1995.0,A family wedding reignites the ancient feud be...,"['Romance', 'Comedy']",0.000000,['English'],11.712900,6.5,92.0,"[{'character': 'Max Goldman', 'name': 'Walter ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,W

In [2]:
# Convertir títulos en una matriz TF-IDF
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(movies['title'])


In [3]:
from sklearn.metrics.pairwise import cosine_similarity

# Calcular la similitud coseno
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)


In [4]:
def recomendar_pelicula(titulo, n_recomendaciones=5):
    try:
        # Buscar índice del título
        idx = movies.index[movies['title'].str.lower() == titulo.lower()].tolist()[0]
        
        # Calcular similitud y ordenar
        sim_scores = list(enumerate(cosine_sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        
        # Tomar las 'n_recomendaciones' más similares
        sim_scores = sim_scores[1:n_recomendaciones + 1]
        indices = [i[0] for i in sim_scores]
        
        return movies['title'].iloc[indices]
    except IndexError:
        return "Película no encontrada. Verifica el título ingresado."


In [5]:
# Prueba de la función
titulo_consulta = "The Matrix"
recomendaciones = recomendar_pelicula(titulo_consulta)
print(f"Películas similares a '{titulo_consulta}':")
print(recomendaciones)


Películas similares a 'The Matrix':
6227       The Matrix Reloaded
6789    The Matrix Revolutions
0                    Toy Story
1                      Jumanji
2             Grumpier Old Men
Name: title, dtype: object


In [7]:
#1. Validación del Modelo
#Para verificar la calidad de las recomendaciones, usaremos una métrica como el Precision@K, que mide la proporción de 
#películas relevantes entre las recomendadas.

def recomendar_con_genero(titulo, genero, n=5):
    """
    Recomienda películas similares al título dado, filtradas por género.
    :param titulo: Título de la película de referencia
    :param genero: Género para filtrar
    :param n: Número de películas a recomendar
    """
    idx = movies.index[movies['title'] == titulo].tolist()[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Filtrar por género
    recommendations = []
    for movie_idx, _ in sim_scores[1:]:
        if genero in movies['genres'].iloc[movie_idx]:
            recommendations.append(movies['title'].iloc[movie_idx])
            if len(recommendations) == n:
                break
    return recommendations

# Ejemplo de uso
recomendaciones = recomendar_con_genero("The Matrix", "Action", n=5)
print("Recomendaciones (con género):", recomendaciones)



Recomendaciones (con género): ['The Matrix Reloaded', 'The Matrix Revolutions', 'Heat', 'Tom and Huck', 'Sudden Death']


In [9]:
# Filtrar por género
def recomendar_por_genero(genero, n=5):
    """
    Recomienda películas basadas en un género.
    :param genero: Género a filtrar
    :param n: Número de películas a recomendar
    """
    filtradas = movies[movies['genres'].str.contains(genero, na=False)]
    return filtradas['title'].head(n).tolist()

# Ejemplo de uso
recomendaciones_genero = recomendar_por_genero("Drama", n=5)
print("Recomendaciones basadas en género:", recomendaciones_genero)


Recomendaciones basadas en género: ['Waiting to Exhale', 'Heat', 'Tom and Huck', 'The American President', 'Nixon']


In [10]:
pip install streamlit


Note: you may need to restart the kernel to use updated packages.Collecting streamlit
  Obtaining dependency information for streamlit from https://files.pythonhosted.org/packages/c2/87/b2e162869500062a94dde7589c167367b5538dab6eacce2e7c0f00d5c9c5/streamlit-1.41.1-py2.py3-none-any.whl.metadata
  Obtaining dependency information for altair<6,>=4.0 from https://files.pythonhosted.org/packages/aa/f3/0b6ced594e51cc95d8c1fc1640d3623770d01e4969d29c0bd09945fafefa/altair-5.5.0-py3-none-any.whl.metadata
  Obtaining dependency information for blinker<2,>=1.0.0 from https://files.pythonhosted.org/packages/10/cb/f2ad4230dc2eb1a74edf38f1a38b9b52277f75bef262d8908e60d957e13c/blinker-1.9.0-py3-none-any.whl.metadata
  Obtaining dependency information for cachetools<6,>=4.0 from https://files.pythonhosted.org/packages/a4/07/14f8ad37f2d12a5ce41206c21820d8cb6561b728e51fad4530dff0552a67/cachetools-5.5.0-py3-none-any.whl.metadata
  Obtaining dependency information for protobuf<6,>=3.20 from https://files.pyt


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
